In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dropout, Flatten, Dense, Input
from tensorflow.keras.applications import VGG16
from tensorflow.keras import optimizers
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [7]:
train_data_dir = '../data/train'
validation_data_dir = '../data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 30
batch_size = 20

# dimensions of our images.
img_width, img_height = 150, 150

test_dir = '../data/test'
test_datagen = ImageDataGenerator(rescale=1./255)



#
# Save bottleneck features from VGG
#

# build the VGG16 network, leaving off the top classifier layer
# so we just get the features as output
print('Building VGG16...')
input_tensor = Input(shape=(img_width, img_height, 3))
base_model = VGG16(weights='imagenet', include_top=False,
        input_tensor=input_tensor)

# Save training features
datagen = ImageDataGenerator(rescale=1/255)
generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None, # class mode set to None here, because images are loaded in order
    shuffle=False)

print('Saving bottleneck features (train)...')
bottleneck_features_train = base_model.predict_generator(
    generator, nb_train_samples // batch_size)
np.save('../features/bottleneck_features_train.npy', bottleneck_features_train)

# Save validation features
generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

print('Saving bottleneck features (validation)...')
bottleneck_features_validation = base_model.predict_generator(
    generator)

np.save('../features/bottleneck_features_validation.npy',
        bottleneck_features_validation)

Building VGG16...


FileNotFoundError: [Errno 2] No such file or directory: '../data/train'

In [8]:
epochs = 30
batch_size = 20
def createHeadLayers(shape, name):
    # setup params and where to save features
    
    # Build model
    top_model = Sequential()
    # flatten the output convolutions, some implementations also 
    #. perform an average pooling here to collapse the features down
    top_model.add(Flatten(shape))
    # add two fully connected layers and some dropout
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dropout(0.5))
    top_model.add(Dense(1, activation='sigmoid'))
    
    top_model.name = name
    
    return top_model

In [10]:
def createMultiTaskModel(inputShape, tasks, input_feature_shape):
    
    # construct both heads
    inputs = Input(shape=inputShape)
    outputs = []
    for i in range(len(tasks)):
        outputs = [outputs, createHeadLayers(input_feature_shape).output]
        
    model = Model(
            inputs=inputs,
            outputs=outputs,
            name="pokemon")
        
    return model

In [11]:
EPOCHS = 30
INIT_LR = 1e-3
BS = 20

# Load bottleneck features training
train_data = np.load('../features/train.npy')
input_shape_features=train_data.shape[1:]

# Load bottleneck features validation
validation_data = np.load('../features/validation.npy')

model = createMultiTaskModel(inputShape, tasks, input_shape_features)

opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=["accuracy"])
history = model.fit(train_data, train_labels,
              epochs=EPOCHS,
              batch_size=BS,
              validation_data=(validation_data, validation_labels),
              verbose=1)

FileNotFoundError: [Errno 2] No such file or directory: '../features/train.npy'

In [12]:
# Plot training and validation accuracy

def plot_training_validation_acc(history, smooth=False, smooth_factor=0.8):
    def smooth_curve(points, factor=0.8):
        smoothed_points = []
        for point in points:
            if smoothed_points:
                previous = smoothed_points[-1]
                smoothed_points.append(previous * factor + point * (1 - factor))
            else:
                smoothed_points.append(point)
        return smoothed_points

    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    if smooth:
        acc = smooth_curve(acc)
        val_acc = smooth_curve(val_acc)
        loss = smooth_curve(loss)
        val_loss = smooth_curve(val_loss)

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()
    
    
plot_training_validation_acc(history)

NameError: name 'history' is not defined